In [1]:
import sys
import logging, os
from time import time
import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
%matplotlib notebook

import ipyparallel as ipp

from bem import Electrodes, Sphere, Cylinder, Mesh, Grid, Configuration, Result
from bem.formats import stl
from bem.bemColors_lib.bemColors import bemColors
from helper_functions import helper_functions
from collections import OrderedDict

In [2]:
prefix = "Calcium_trap_planar_cood_fixed v5"

In [3]:
scale = 1e-8    # rescale factor 1 um.
use_stl = True

# load electrode faces from colored stl
# s_nta is intermediate processed stl file.
s_nta = stl.read_stl(open("%s.stl" % prefix, "rb"))
print("Import stl:",os.path.abspath("./"+prefix+".stl"),"\n")
print("Electrode colors (numbers):\n")
mesh = Mesh.from_mesh(stl.stl_to_mesh(*s_nta, scale=scale/1e-6,rename={1:"1"})) # unit inside is um

Import stl: c:\Users\Photonics\Documents\GitHub\integrated_photonics_bem\Weiwei_simulation\Calcium_trap_planar_cood_fixed v5.stl 

Electrode colors (numbers):

dropping 20083
dropping 5345
dropping 10095
dropping 19501
dropping 15925
dropping 7275
dropping 13683
dropping 1647
dropping 13363
dropping 28135
dropping 19943
dropping 11517
dropping 28463
dropping 1399
dropping 10151
dropping 15669
dropping 5493
dropping 11893
dropping 5997
dropping 5941
dropping 27949
dropping 9893
dropping 3565
dropping 15905
dropping 25969
mesh.gather() not working properly, be sure to have valid input


In [4]:
ele_col = bemColors(np.array(list(set(s_nta[2]))),('fusion360','export_stl'))
ele_col.set_my_color(value = (178,178,178),cl_format = ('fusion360','export_stl','RGBA64'),name = 'self_defined')
ele_col.print_stl_colors()

COLORS in the stl:
['bem6']
['bem13']
['bem19']
['bem17']
['bem11']
['bem15']
['bem1']
['bem22']
['bem12']
['bem5']
['bem18']
['bem3']
['bem16']
['bem7']
['bem9']
['bem23']
['bem4']
['bem2']
['bem20']
['bem10']
['bem14']
['bem24']
['bem8']
['bem21']
['_unkColor0']
TOTAL COLORS:  25


In [5]:
# # assign a name for each color
ele_col.color_electrode(color = 'bem1',name = 'cutout1')
ele_col.color_electrode(color = 'bem2',name = 'cutout2')
ele_col.color_electrode(color = 'bem3',name = 'GND')
ele_col.color_electrode(color = 'bem4',name = 'DC1')
ele_col.color_electrode(color = 'bem5',name = 'DC2')
ele_col.color_electrode(color = 'bem6',name = 'DC3')
ele_col.color_electrode(color = 'bem7',name = 'DC4')
ele_col.color_electrode(color = 'bem8',name = 'DC5')
ele_col.color_electrode(color = 'bem9',name = 'DC6')
ele_col.color_electrode(color = 'bem10',name = 'DC7')
ele_col.color_electrode(color = 'bem11',name = 'DC8')
ele_col.color_electrode(color = 'bem12',name = 'DC9')
ele_col.color_electrode(color = 'bem13',name = 'DC10')
ele_col.color_electrode(color = 'bem14',name = 'RF')
ele_col.color_electrode(color = 'bem15',name = 'DC11')
ele_col.color_electrode(color = 'bem16',name = 'DC12')
ele_col.color_electrode(color = 'bem17',name = 'DC13')
ele_col.color_electrode(color = 'bem18',name = 'DC14')
ele_col.color_electrode(color = 'bem19',name = 'DC15')
ele_col.color_electrode(color = 'bem20',name = 'DC16')
ele_col.color_electrode(color = 'bem21',name = 'DC17')
ele_col.color_electrode(color = 'bem22',name = 'DC18')
ele_col.color_electrode(color = 'bem23',name = 'DC19')
ele_col.color_electrode(color = 'bem24',name = 'DC20')

# # print colors still with no name. These meshes will be neglected in the following codes
ele_col.drop_colors()

# # read stl into mesh with electrode names
# # unnamed meshes will not be imported at all
mesh = Mesh.from_mesh(stl.stl_to_mesh(*s_nta, scale=scale/1e-6,
    rename=ele_col.electrode_colors, quiet=False))

dropping color ['_unkColor0']
TOTAL COLORS DROPPED:  1
dropping 20083
1 planes in electrode DC14
normals vectors:
 [[0. 1. 0.]]
1 planes in electrode DC1
normals vectors:
 [[ 8.18265803e-16  1.00000000e+00 -0.00000000e+00]]
1 planes in electrode DC13
normals vectors:
 [[0. 1. 0.]]
1 planes in electrode DC18
normals vectors:
 [[0. 1. 0.]]
1 planes in electrode DC4
normals vectors:
 [[8.18265803e-16 1.00000000e+00 0.00000000e+00]]
1 planes in electrode DC7
normals vectors:
 [[ 8.18265803e-16  1.00000000e+00 -0.00000000e+00]]
41 planes in electrode cutout2
normals vectors:
 [[ 8.75317097e-01  0.00000000e+00 -4.83549327e-01]
 [ 8.87170017e-01  0.00000000e+00 -4.61442709e-01]
 [ 8.98672283e-01  0.00000000e+00 -4.38620657e-01]
 [ 9.09385741e-01  0.00000000e+00 -4.15953815e-01]
 [ 9.19606209e-01  0.00000000e+00 -3.92841488e-01]
 [ 9.29189205e-01  0.00000000e+00 -3.69604468e-01]
 [ 9.38208878e-01  0.00000000e+00 -3.46069455e-01]
 [ 9.46700871e-01  0.00000000e+00 -3.22113991e-01]
 [ 9.54511762e

In [6]:
mesh.plot()

In [7]:
print(len(mesh.points))

1004


In [8]:
mesh.triangulate(opts="q20Q",new = False);

start triangulate DC14
final opts q20Qzr
finish triangulate DC14
start triangulate DC1
final opts q20Qzr
finish triangulate DC1
start triangulate DC13
final opts q20Qzr
finish triangulate DC13
start triangulate DC18
final opts q20Qzr
finish triangulate DC18
start triangulate DC4
final opts q20Qzr
finish triangulate DC4
start triangulate DC7
final opts q20Qzr
finish triangulate DC7
start triangulate cutout2
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20

In [9]:
mesh.plot()

In [10]:
print(len(mesh.points))

4527


In [11]:
mesh.areas_from_constraints(Sphere(center=np.array([0.0, 0.0, -9.0]),
           radius=5, inside=0.01, outside=1.))    # "inside", "outside" set different mesh densities.
# mesh.areas_from_constraints(Sphere(center=np.array([0., 0., 1]),
#            radius=3.5, inside=0.1, outside=5))    # "inside", "outside" set different mesh densities.
# retriangulate quality and quiet with areas
mesh.triangulate(opts="q20Q", new=False)
# save base mesh to vtk
mesh.to_vtk(prefix)
print("Output vtk:",os.path.abspath("./"+prefix+".vtk"))    # output path


start triangulate DC14
final opts q20Qzra
finish triangulate DC14
start triangulate DC1
final opts q20Qzra
finish triangulate DC1
start triangulate DC13
final opts q20Qzra
finish triangulate DC13
start triangulate DC18
final opts q20Qzra
finish triangulate DC18
start triangulate DC4
final opts q20Qzra
finish triangulate DC4
start triangulate DC7
final opts q20Qzra
finish triangulate DC7
start triangulate cutout2
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20

In [12]:
mesh.plot()

In [13]:
print(len(mesh.points))

9768


In [14]:
def run_job(args):
    # job is Configuration instance.
    job, grid, prefix = args
    # refine twice adaptively with increasing number of triangles, min angle 15 deg.
#     job.adapt_mesh(triangles=4e2, opts="q0.1Q")
#     job.adapt_mesh(triangles=1e3, opts="q0.1Q")
    # solve for surface charges
    job.solve_singularities(num_mom=4, num_lev=3, max_iter=400)
    # get potentials and fields
    # RF_field = (job.name=="RF1") or (job.name=="RF2") or (job.name=="RF0")
    RF_field = "RF" in job.name
    result = job.simulate(grid, field=RF_field, num_lev=2)    # For "RF", field=True computes the field.
    result.to_vtk(prefix)
    print("finished job %s" % job.name)
    return job.collect_charges()

In [15]:
# grid to evalute potential and fields atCreate a grid in unit of scaled length l. Only choose the interested region (trap center) to save time.
s = 0.005 # step 0.01*100 (scale) = 1 um
Lx, Ly, Lz = 0.6, 0.3, 0.2   # in the unit of scaled length l, z=0 is the bottom of substrate, z = 30um is the top of metal
sx, sy, sz = s, s, s
# ni is grid point number, si is step size. Thus to fix size on i direction you need to fix ni*si.
# nx, ny, nz = [2*np.ceil(L/2.0/s).astype('int')+1 for L in (Lx, Ly, Lz)]
nx = 2*np.ceil(Lx/2.0/sx).astype('int')+1 
ny = 2*np.ceil(Ly/2.0/sy).astype('int')+1 
nz = 2*np.ceil(Lz/2.0/sz).astype('int')+1 
print("Size/l:", Lx, Ly, Lz)
print("Step/l:", sx, sy, sz)
print("Shape (grid point numbers):", nx, ny, nz)
grid = Grid(center=(0., 0.0, -9.0), step=(sx, sy, sz), shape=(nx, ny, nz))
# Grid center (nx, ny ,nz)/2 is shifted to origin
print("Grid origin/l:", grid.get_origin()[0])
x, y, z = grid.to_xyz()

Size/l: 0.6 0.3 0.2
Step/l: 0.005 0.005 0.005
Shape (grid point numbers): 121 61 41
Grid origin/l: -0.3


In [16]:
pv.global_theme.notebook = False
def plot_grid_mesh_stl(grid, mesh):
    plotter = pv.Plotter()
    
    # Convert the mesh to PolyData and add it to the plotter
    pd = mesh.to_polydata()
    plotter.add_mesh(pd, scalars='electrode_name', show_edges=True)
    
    # Add the grid points
    x, y, z = np.meshgrid(grid.to_xyz()[0], grid.to_xyz()[1], grid.to_xyz()[2])
    grid_points = np.vstack([x.ravel(), y.ravel(), z.ravel()]).T
    plotter.add_points(grid_points, color='red', point_size=5)
    
    plotter.show_bounds()
    plotter.add_camera_orientation_widget()
    plotter.show()

# Call the function to plot everything together
plot_grid_mesh_stl(grid, mesh)

In [17]:
jobs = list(Configuration.select(mesh, "RF"))    # select() picks one electrode each time.
# jobs = list(Configuration.select(mesh, "RF"))    # select() picks one electrode each time.
# jobs = list(Configuration.select(mesh, "DC.*")) 



# parallel computation
mycluster = ipp.Cluster()
mycluster.start_cluster_sync()
c = mycluster.connect_client_sync()
c.wait_for_engines()

t0 = time()
# Run a parallel map, executing the wrapper function on indices 0,...,n-1
lview = c.load_balanced_view()
# Cause execution on main process to wait while tasks sent to workers finish
lview.block = True 
asyncresult = lview.map_async(run_job, ((job, grid, prefix) for job in jobs))   # Run calculation in parallel
asyncresult.wait_interactive()
print("Computing time: %f s"%(time()-t0))

Starting 32 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/32 [00:00<?, ?engine/s]

run_job:   0%|          | 0/1 [00:00<?, ?tasks/s]

Computing time: 553.246155 s


In [18]:
print(len(mesh.points))

9768


In [19]:
import ipyparallel as ipp
from time import time
import psutil

# Select jobs
jobs = list(Configuration.select(mesh, "DC"))

# Parallel computation setup
mycluster = ipp.Cluster()
mycluster.start_cluster_sync()
c = mycluster.connect_client_sync()
c.wait_for_engines()

# Limit engines to 30
selected_engines = c.ids[:30] if len(c.ids) > 30 else c.ids

# Track memory usage function
def check_memory():
    mem = psutil.virtual_memory()
    print(f"Memory Usage: {mem.percent}% - Available: {mem.available / 1e9:.2f} GB")

# Create a direct view on selected engines
lview = c.direct_view(targets=selected_engines)
lview.block = True

# Run calculation in parallel
t0 = time()
check_memory()

# Run in smaller batches to avoid memory overload
batch_size = min(100, len(jobs))  # Adjust batch size as needed
for i in range(0, len(jobs), batch_size):
    asyncresult = lview.map_async(run_job, ((job, grid, prefix) for job in jobs[i:i + batch_size]))
    asyncresult.wait_interactive()
    check_memory()

print("Computing time: %f s" % (time() - t0))


Starting 32 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/32 [00:00<?, ?engine/s]

Memory Usage: 37.2% - Available: 43.01 GB


run_job:   0%|          | 0/20 [00:00<?, ?tasks/s]

Memory Usage: 13.8% - Available: 59.00 GB
Computing time: 828.921382 s


In [20]:
jobs = list(Configuration.select(mesh, "DC"))
for job in jobs:
    run_job((job, grid, prefix))

c:\Users\Photonics\.conda\envs\bem39\lib\site-packages\pyvista\core\grid.py:514: PyVistaDeprecationWarning: `dims` argument is deprecated. Please use `dimensions`.
  warnings.warn(


finished job DC14


c:\Users\Photonics\.conda\envs\bem39\lib\site-packages\pyvista\core\grid.py:514: PyVistaDeprecationWarning: `dims` argument is deprecated. Please use `dimensions`.
  warnings.warn(


finished job DC1


c:\Users\Photonics\.conda\envs\bem39\lib\site-packages\pyvista\core\grid.py:514: PyVistaDeprecationWarning: `dims` argument is deprecated. Please use `dimensions`.
  warnings.warn(


...

c:\Users\Photonics\.conda\envs\bem39\lib\site-packages\pyvista\core\grid.py:514: PyVistaDeprecationWarning: `dims` argument is deprecated. Please use `dimensions`.
  warnings.warn(


finished job DC19


c:\Users\Photonics\.conda\envs\bem39\lib\site-packages\pyvista\core\grid.py:514: PyVistaDeprecationWarning: `dims` argument is deprecated. Please use `dimensions`.
  warnings.warn(


finished job DC3
finished job DC16


c:\Users\Photonics\.conda\envs\bem39\lib\site-packages\pyvista\core\grid.py:514: PyVistaDeprecationWarning: `dims` argument is deprecated. Please use `dimensions`.
  warnings.warn(


In [21]:
jobs = list(Configuration.select(mesh, "DC"))    # select() picks one electrode each time.
# jobs = list(Configuration.select(mesh, "RF"))    # select() picks one electrode each time.
# jobs = list(Configuration.select(mesh, "DC.*")) 



# parallel computation
mycluster = ipp.Cluster()
mycluster.start_cluster_sync()
c = mycluster.connect_client_sync()
c.wait_for_engines()

t0 = time()
# Run a parallel map, executing the wrapper function on indices 0,...,n-1
lview = c.load_balanced_view()
# Cause execution on main process to wait while tasks sent to workers finish
lview.block = True 
asyncresult = lview.map_async(run_job, ((job, grid, prefix) for job in jobs))   # Run calculation in parallel
asyncresult.wait_interactive()
print("Computing time: %f s"%(time()-t0))

Starting 32 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/32 [00:00<?, ?engine/s]

run_job:   0%|          | 0/20 [00:00<?, ?tasks/s]

Computing time: 811.607075 s


In [22]:
Result.view(prefix, 'DC1')

In [23]:
Result.view(prefix, 'RF')

In [ ]:
jobs = list(Configuration.select(mesh, "GND"))    # select() picks one electrode each time.
# jobs = list(Configuration.select(mesh, "RF"))    # select() picks one electrode each time.
# jobs = list(Configuration.select(mesh, "DC.*")) 

# parallel computation
mycluster = ipp.Cluster()
mycluster.start_cluster_sync()
c = mycluster.connect_client_sync()
c.wait_for_engines()

t0 = time()
# Run a parallel map, executing the wrapper function on indices 0,...,n-1
lview = c.load_balanced_view()
# Cause execution on main process to wait while tasks sent to workers finish
lview.block = True 
asyncresult = lview.map_async(run_job, ((job, grid, prefix) for job in jobs))   # Run calculation in parallel
asyncresult.wait_interactive()
print("Computing time: %f s"%(time()-t0))

Starting 32 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/32 [00:00<?, ?engine/s]

run_job:   0%|          | 0/1 [00:00<?, ?tasks/s]

Computing time: 131.516791 s


In [25]:
Result.view(prefix, 'GND')

In [31]:
Result.view(prefix, 'GND')